<a href="https://colab.research.google.com/github/mozey256/OSCAAR/blob/main/ML_part2_Chem_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 24.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
#---------------------- RDKit packages
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit import DataStructs
import matplotlib.pyplot as plt
from rdkit.ML.Cluster import Butina
#------------------- progress bar
from tqdm import tqdm
#------------------- hide warning
import warnings
warnings.filterwarnings('ignore')

In [3]:
data1 = pd.read_csv('/content/filledmissing_chemo_data.csv')

In [4]:
from scipy.stats import zscore
# Calculate Z-scores for each column
z_scores_df = data1.apply(zscore)

# Save the Z-scores DataFrame to a CSV file
z_scores_df.to_csv('z_scores_file.csv', index=False)


In [5]:
z_data = pd.read_csv('/content/z_scores_file.csv')

In [6]:
z_data.head()

,2aminobenzothiazole_conc3_T30,2hydroxyethylhydrazine_conc3_T30,3aminotriazole_conc3_T30,4hydroxytamoxifene_conc3_T30,A23187_conc3_T30,abietic-acid_conc3_T30,acifluorofen-methyl_conc3_T30,aconitine_conc3_T30,agelasine_conc3_T30,alamethicin_conc3_T30,...,thiabendazole_conc3_T30,thozonium-bromide_conc3_T30,tomatine_conc3_T30,trichostatinA_conc3_T30,trimethoprim_conc3_T30,tunicamycin_conc3_T30,usnic-acid_conc3_T30,valinomycin_conc3_T30,verrucarin_conc3_T30,ZnCl2_conc3_T30
0,-0.276245,2.380651e+00,-1.080706,-0.215067,0.154464,-7.752147e-01,-0.444465,-3.410281e-01,1.348328e+00,3.910170e+00,...,-0.682448,1.629095,0.245411,-6.144045e+00,1.088026,1.270172,-0.776692,-0.053471,1.376039e+00,2.986627
1,0.907443,5.214971e-18,0.365249,-7.828689,2.983907,-9.300495e-18,-1.958781,8.285491e-18,-1.108944e+01,1.616172e-17,...,1.019464,0.113800,0.472077,-1.074468e-17,2.114776,-0.010985,-2.232127,0.000000,1.675731e-17,-4.357585
2,0.070030,5.214971e-18,2.548618,-3.659564,4.797881,-9.300495e-18,1.580769,8.285491e-18,2.068249e-18,1.616172e-17,...,0.369397,4.756480,-0.442058,-1.074468e-17,-0.928683,-3.357684,0.932817,0.000000,1.675731e-17,-6.113393
3,-0.793781,-1.654389e+00,-2.457958,1.286591,-3.646740,4.475250e-01,-0.802185,-4.796525e-01,1.408856e+00,2.107696e+00,...,0.789122,-1.796334,-2.397509,-3.316026e+00,-0.570866,-0.740249,3.613389,-4.616956,-8.072135e-01,-3.772715
4,-1.344561,2.955213e+00,1.547518,0.732225,-0.235880,1.118344e+00,-1.510228,-1.277470e+00,-1.241691e-01,3.938488e-01,...,-0.008011,-0.080713,-1.370630,4.120494e+00,-0.846794,0.745750,-0.101235,-1.065809,1.410540e+00,0.297742


In [7]:
z_score_data = z_scores_df.values

In [8]:
#the numarry is a numpy arry
modified_array = np.where((z_score_data <= -2) | (z_score_data >=2), 1, 0)

print( modified_array)

#convert the numpy array to dataframe
chem_binorized = pd.DataFrame(modified_array)



[[0 1 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [9]:
chem_binorized.to_csv("binary_zscore.csv")

In [10]:
chem_binorized.head()

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,1
2,0,0,1,1,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,0,0,1,0,1,0,0,0,0,1,...,0,0,1,1,0,0,1,1,0,1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [11]:
zdata = pd.read_csv("./binary_zscore.csv")

In [12]:
zdata.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,139,140,141,142,143,144,145,146,147,148
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,1,0,1,0,0,1
2,2,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,3,0,0,1,0,1,0,0,0,0,...,0,0,1,1,0,0,1,1,0,1
4,4,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
zdata2 = zdata.drop(columns=['Unnamed: 0'])

In [14]:
zbinary=zdata2

In [15]:
columns = ["2aminobenzothiazole_conc3_T30",
           "2hydroxyethylhydrazine_conc3_T30",	"3aminotriazole_conc3_T30",	"4hydroxytamoxifene_conc3_T30", "A23187_conc3_T30", "abietic-acid_conc3_T30",	"acifluorofen-methyl_conc3_T30",	"aconitine_conc3_T30",	"agelasine_conc3_T30",	"alamethicin_conc3_T30",	"alexidine_conc3_T30",
           "allantoin_conc3_T30",	"alternariol_conc3_T30",	"aluminum-sulfate_conc3_T30",	"amantadine_conc3_T30",	"amiodarone_conc3_T30",
           "ammonium-persulfate_conc3_T30",	"amphotericinB_conc3_T30", "andrastin_conc3_T30",	"antimycin_conc3_T30",	"apicidin_conc3_T30",	"artemesinin_conc3_T30",
           "azide_conc3_T30",	"BaCl2_conc3_T30",	"bafilomycin_conc3_T30",	"BCS_conc3_T30",	"betulinic-acid_conc3_T30",
           "bifonazole_conc3_T30",	"borate_conc3_T30",	"brefeldinA_conc3_T30",	"CaCl2_conc3_T30",	"caffeine_conc3_T30",
           "calcofluor-white_conc3_T30",	"camptothecin_conc3_T30",	"castanospermine_conc3_T30",	"cerulenin_conc3_T30",
           "chloroquine_conc3_T30",	"chlorpromazine_conc3_T30",	"chromium-III_conc3_T30",	"ciclopirox-olamine_conc3_T30",
           "cisplatin_conc3_T30",	"climbazole_conc3_T30",	"clotrimazole_conc3_T30",	"colistin_conc3_T30",	"congo-red_conc3_T30",
           "crystal-violet_conc3_T30",	"cycloheximide_conc3_T30",	"cyclosporin_conc3_T30",	"cyclosporin-FK506_conc3_T30",
           "cyproconazole_conc3_T30",	"daphnetin_conc3_T30",	"deferoxamine_conc3_T30",	"desiperimine_conc3_T30",
           "dyclonine_conc3_T30",	"emodin_conc3_T30",	"erlotinib_conc3_T30", "fenpropimorph_conc3_T30",	"fenpropimorph-fluconazole_conc3_T30",
           "FK506_conc3_T30",	"fluconazole_conc2_T30","fluorocytosine_conc3_T30",	"fluorocytosine-2_conc3_T30",	"fluspirilene_conc3_T30",	"gallium-nitrate_conc3_T30",
           "geldanamycin_conc3_T30",	"haloperidol_conc3_T30",	"hydrogen-peroxide_conc3_T30",	"hydroxyurea_conc3_T30",	"hygromycin_conc3_T30",
           "imazilil_conc3_T30",	"itraconazole_conc3_T30",	"K252a_conc3_T30",	"latrunculin_conc3_T30",	"leptomycin_conc3_T30",
           "LiCl_conc3_T30",	"lovastatin_conc3_T30",	"malachite-green_conc3_T30",	"manganese-sulfate_conc3_T30",	"mastaparan_conc3_T30",
           "menadione_conc3_T30",	"menthol_conc3_T30",	"metavanadate_conc3_T30",	"methotrexate_conc3_T30",	"methylbenzethonium_conc3_T30",
           "MG132_conc3_T30",	"miconazole_conc3_T30",	"mitomycin_conc3_T30",	"MMS-1_conc3_T30",	"MMS-2_conc3_T30",	"MMS-3_conc3_T30",
           "MMS-4_conc3_T30",	"MMS-5_conc3_T30"	"MMS-6_conc3_T30",	"MMS-7_conc3_T30",	"myclobutanil_conc3_T30",	"mycophenolic-acid_conc3_T30",
           "NA8_conc3_T30",	"NaCl_conc3_T30",	"NaNO2_conc3_T30",	"neomycin-sulfate_conc3_T30",	"nicotinamide_conc3_T30",	"nigericin_conc3_T30",
           "nikkomycin_conc3_T30",	"NiSO4_conc3_T30",	"ophiobolin_conc3_T30",	"palmitic-acid_conc3_T30",	"parthenolie_conc3_T30",
           "pentamidine_isethionate_conc3_T30",	"pH_conc3_T30",	"phenanthroline_conc3_T30",	"polyoxinB_conc3_T30",	"Prussian-Blue_conc3_T30",
           "quinic-acid_conc3_T30",	"rapamycin_conc3_T30",	"rapamycin-2_conc3_T30",	"rapamycin-3_conc3_T30",	"rapamycin-4_conc3_T30",
           "rapamycin-5_conc3_T30",	"rapamycin-6_conc3_T30",	"rapamycin-7_conc3_T30",	"rapamycin-and-geldanamycin_conc3_T30",
           "rifamycin_conc3_T30",	"S10_conc3_T30",	"S8_conc3_T30",	"SDS_conc3_T30",	"selumenitib_conc3_T30",	"sertraline_conc3_T30",
           "sertraline-fluconazole_conc3_T30",	"sodium-iodide_conc3_T30",	"sodium-molybdate_conc3_T30",	"sodium-selenite_conc3_T30",
           "sodium-sulfite_conc3_T30", "sorafenib_conc3_T30",	"STF-62247_conc3_T30",	"suloctodil_conc3_T30",	"tamoxifen-citrate_conc3_T30",
           "tautomycin_conc3_T30",	"tellurite_conc3_T30",	"terbinafine_conc3_T30",	"thiabendazole_conc3_T30",	"thozonium-bromide_conc3_T30",
           "tomatine_conc3_T30",	"trichostatinA_conc3_T30",	"trimethoprim_conc3_T30",	"tunicamycin_conc3_T30",	"usnic-acid_conc3_T30",
           "valinomycin_conc3_T30",	"verrucarin_conc3_T30",	"ZnCl2_conc3_T30"]

# Creating a dictionary for column mapping with numerical indices as keys
column_mapping = {str(index): column for index, column in enumerate(columns)}

# Assuming 'data' is your DataFrame
# Use the rename method to apply the column name changes
zbinary.rename(columns=column_mapping, inplace=True)


In [16]:
zbinary.head()

,2aminobenzothiazole_conc3_T30,2hydroxyethylhydrazine_conc3_T30,3aminotriazole_conc3_T30,4hydroxytamoxifene_conc3_T30,A23187_conc3_T30,abietic-acid_conc3_T30,acifluorofen-methyl_conc3_T30,aconitine_conc3_T30,agelasine_conc3_T30,alamethicin_conc3_T30,...,thozonium-bromide_conc3_T30,tomatine_conc3_T30,trichostatinA_conc3_T30,trimethoprim_conc3_T30,tunicamycin_conc3_T30,usnic-acid_conc3_T30,valinomycin_conc3_T30,verrucarin_conc3_T30,ZnCl2_conc3_T30,148
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,1
2,0,0,1,1,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,0,0,1,0,1,0,0,0,0,1,...,0,0,1,1,0,0,1,1,0,1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [17]:
zbinary.to_csv("Drug_chemo_zscores.csv")

In [18]:
zbina2 = pd.read_csv('/content/Drug_chemo_zscores.csv')

In [33]:
zbina2.head()

,2aminobenzothiazole_conc3_T30,2hydroxyethylhydrazine_conc3_T30,3aminotriazole_conc3_T30,4hydroxytamoxifene_conc3_T30,A23187_conc3_T30,abietic-acid_conc3_T30,acifluorofen-methyl_conc3_T30,aconitine_conc3_T30,agelasine_conc3_T30,alamethicin_conc3_T30,...,thiabendazole_conc3_T30,thozonium-bromide_conc3_T30,tomatine_conc3_T30,trichostatinA_conc3_T30,trimethoprim_conc3_T30,tunicamycin_conc3_T30,usnic-acid_conc3_T30,valinomycin_conc3_T30,verrucarin_conc3_T30,ZnCl2_conc3_T30
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
2,0,0,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,0,0,1,0,1,0,0,0,0,1,...,0,0,0,1,1,0,0,1,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
zbina2 = zbina2.drop(columns=['148'])

In [36]:
binary = zbina2.T

In [31]:
zbina2 = zbina2.drop(columns=['Unnamed: 0'])

KeyError: "['Unnamed: 0'] not found in axis"

In [37]:
binary.head()

,0,1,2,3,4,5,6,7,8,9,...,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451
2aminobenzothiazole_conc3_T30,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1
2hydroxyethylhydrazine_conc3_T30,1,0,0,0,1,0,1,1,1,0,...,0,0,0,1,0,0,0,0,1,1
3aminotriazole_conc3_T30,0,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
4hydroxytamoxifene_conc3_T30,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
A23187_conc3_T30,0,1,1,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1


In [38]:
binary.tail()

,0,1,2,3,4,5,6,7,8,9,...,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451
tunicamycin_conc3_T30,0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,1
usnic-acid_conc3_T30,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,1
valinomycin_conc3_T30,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
verrucarin_conc3_T30,0,0,0,1,0,0,1,1,0,1,...,1,0,0,0,0,0,0,0,1,1
ZnCl2_conc3_T30,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,1,1


In [39]:
binary = binary.reset_index(drop=False)

In [40]:
binary.head()

,index,0,1,2,3,4,5,6,7,8,...,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451
0,2aminobenzothiazole_conc3_T30,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1
1,2hydroxyethylhydrazine_conc3_T30,1,0,0,0,1,0,1,1,1,...,0,0,0,1,0,0,0,0,1,1
2,3aminotriazole_conc3_T30,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
3,4hydroxytamoxifene_conc3_T30,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,A23187_conc3_T30,0,1,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1


In [41]:
drug_names = binary['index'].tolist()

In [42]:
import itertools
combinations = list(itertools.combinations(drug_names, 2))

for comb in combinations:
  print(f"Combination: {comb[0]} + {comb[1]}")

with open('drug combinations.txt', 'w') as file:
  for comb in combinations:
    file.write(f"{comb[0]} + {comb[1]}\n")

Streaming output truncated to the last 5000 lines.
Combination: cyclosporin_conc3_T30 + neomycin-sulfate_conc3_T30
Combination: cyclosporin_conc3_T30 + nicotinamide_conc3_T30
Combination: cyclosporin_conc3_T30 + nigericin_conc3_T30
Combination: cyclosporin_conc3_T30 + nikkomycin_conc3_T30
Combination: cyclosporin_conc3_T30 + NiSO4_conc3_T30
Combination: cyclosporin_conc3_T30 + ophiobolin_conc3_T30
Combination: cyclosporin_conc3_T30 + palmitic-acid_conc3_T30
Combination: cyclosporin_conc3_T30 + parthenolie_conc3_T30
Combination: cyclosporin_conc3_T30 + pentamidine_isethionate_conc3_T30
Combination: cyclosporin_conc3_T30 + pH_conc3_T30
Combination: cyclosporin_conc3_T30 + phenanthroline_conc3_T30
Combination: cyclosporin_conc3_T30 + polyoxinB_conc3_T30
Combination: cyclosporin_conc3_T30 + Prussian-Blue_conc3_T30
Combination: cyclosporin_conc3_T30 + quinic-acid_conc3_T30
Combination: cyclosporin_conc3_T30 + rapamycin_conc3_T30
Combination: cyclosporin_conc3_T30 + rapamycin-2_conc3_T30
Com

In [43]:
zscores = binary.drop(columns=['index'])
zscores = zscores.astype(int)

joint_zscores ={}

for comb in combinations:
  drug1 , drug2 = comb
  zprofile1 = zscores.loc[binary['index']==drug1].values[0][1:]
  zprofile2 = zscores.loc[binary['index']==drug2].values[0][1:]
  joint_zscore = (zprofile1 + zprofile2)
  joint_zscores[comb]= joint_zscore

In [44]:
zdata = pd.DataFrame(joint_zscores).T
zdata['Combination'] = zdata.index.map(lambda x: "+".join(x))

output_csv_file = 'joint z scores.csv'

zdata = zdata[['Combination'] + [col for col in zdata.columns if col != 'Combination']]

zdata.to_csv('chemo_combination.csv')

In [45]:
dataflu = pd.read_csv('/content/chemo_combination.csv')

In [46]:
datafluco = dataflu

In [48]:
# Filter rows containing 'fluconazole'
df_fluconazole = datafluco[datafluco['Combination'].str.contains('fluconazole_conc2_T30')]

# Write the filtered dataset to a new file
df_fluconazole.to_csv('fluconazole_dataset.csv', index=False)


In [49]:
# Filter rows containing 'geldanamycin'

df_geldanamycin = datafluco[datafluco['Combination'].str.contains('geldanamycin_conc3_T30')]

# Write the filtered dataset to a new file
df_geldanamycin.to_csv('new_geldanamycin.csv', index=False)

In [50]:
dffl = pd.read_csv('/content/fluconazole_dataset.csv')
dfgl = pd.read_csv('/content/new_geldanamycin.csv')

In [53]:
# Specify the common key column(s) based on which you want to join the datasets
common_key = 'Combination'

# Perform the inner join on the common key column(s)
merged_df = pd.merge(dffl, dfgl, on=common_key, how='inner')

# Save the merged dataset to a new CSV file
merged_df.to_csv('merged_flc_gld.csv', index=False)